In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [7]:
#xml to csv

In [2]:
tree = ET.parse('output_network.xml')
root = tree.getroot()

In [3]:
node2geom = {}
for nodes in root.findall('nodes'):
    for node in nodes.findall('node'):
        node2geom[node.get('id')]=[node.get('x'),node.get('y')]

In [4]:
cols=[ 'WKT','link','length','freespeed','capacity','permlanes','oneway','modes']
rows = []

In [5]:
for links in root.findall('links'):
    for link in links.findall('link'):
        f = link.get('from')
        t = link.get('to')

        if f in node2geom.keys() and  t in node2geom.keys():
            geom_f= node2geom[f]
            geom_t= node2geom[t]
            
            wkt = 'MULTILINESTRING (('+geom_f[0]+' '+geom_f[1]+','+geom_t[0]+' '+geom_t[1]+'))'
            
            rows.append({'WKT':wkt, 
                         'link':link.get('id'),
                         'length':link.get('length'),
                         'freespeed':link.get('freespeed'),
                         'capacity':link.get('capacity'),
                         'permlanes':link.get('permlanes'),
                         'oneway':link.get('oneway'),
                         'modes':link.get('modes')})

In [6]:
df = pd.DataFrame(rows, columns=cols)
df.to_csv('output_network.csv',index=False)

In [20]:
#csv to shp , get intersection

```
add index to linestring_bulidings.shp

Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'index', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 1, 'FORMULA' : '@row_number', 'INPUT' : 'G:/805/linestrings_buildings.shp', 'OUTPUT' : 'G:/820_fix_osm_link/linestrings_buildings_index.shp' }

line intersection

Algorithm 'Line intersections' starting…
Input parameters:
{ 'INPUT' : 'G:/820_fix_osm_link/linestrings_buildings_index.shp', 'INPUT_FIELDS' : ['FID','index'], 'INTERSECT' : 'G:/820_fix_osm_link/osm_network.shp', 'INTERSECT_FIELDS' : ['link','freespeed','capacity','permlanes','oneway','modes'], 'INTERSECT_FIELDS_PREFIX' : '', 'OUTPUT' : 'G:/820_fix_osm_link/intersection.shp' }
```

In [ ]:
#intersection shp to csv ,read csv , get osm link

In [149]:
import csv 

In [150]:
dict_link={}
dict_freespeed={}
dict_capacity ={}
dict_permlanes={}
dict_modes = {}
with open ('intersection.csv','rt',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)
    
    for row in reader :
        fid, index, link,freespeed,capacity,permlanes,oneway,modes =row
        #dict_link
        if fid in dict_link.keys():
            if link not in dict_link[fid]:
                dict_link[fid].append(link)
        else:
            dict_link[fid]=[link]
        
        #dict freespeed
        if (fid,index) in dict_freespeed.keys():
            dict_freespeed[(fid,index)].append(freespeed)
        else:
            dict_freespeed[(fid,index)] = [freespeed]              
        
        #dict  capacity
        if (fid,index) in dict_capacity.keys():
            dict_capacity[(fid,index)].append(capacity)
        else:
            dict_capacity[(fid,index)] = [capacity]
            
        #dict permelances
        if (fid,index) in dict_permlanes.keys():
            dict_permlanes[(fid,index)].append(permlanes)
        else:
            dict_permlanes[(fid,index)] = [permlanes]
            
        #dict modes
        if (fid,index) in dict_modes.keys():
            dict_modes[(fid,index)].append(modes)
        else:
            dict_modes[(fid,index)] = [modes]

In [151]:
fid2freespeed= {}
for key ,value in dict_freespeed.items():
    fid, index = key  
    c=0
    for v in value:
        c= c+ float(v)
    c= c/len(value)
    if fid in fid2freespeed.keys():
        fid2freespeed[fid].append(c)
    else:
        fid2freespeed[fid]=[c]


fid2capacity= {}
for key ,value in dict_capacity.items():
    fid, index = key
    
    c=0
    for v in value:
        c= c+ float(v)
    
    if fid in fid2capacity.keys():
        fid2capacity[fid].append(c)
    else:
        fid2capacity[fid]=[c]
    
fid2permlanes= {}
for key ,value in dict_permlanes.items():
    fid, index = key
    
    c=0
    for v in value:
        c= c+ float(v)
    
    if fid in fid2permlanes.keys():
        fid2permlanes[fid].append(c)
    else:
        fid2permlanes[fid]=[c]
        
fid2modes= {}
for key ,value in dict_modes.items():
    fid, index = key
    for v in value:
        for m in v.split(','):
            if fid in fid2modes.keys():
                
                if m not in fid2modes[fid]:
                    fid2modes[fid].append(m)
            else:
                fid2modes[fid]=[m]    
                
                
fid2link={}
for key,value in dict_link.items():
    fid2link[key]=' '.join(value)

In [152]:
avg_freespeed = {}
for key ,value in fid2freespeed.items():
    avg_freespeed[key] = sum(value)/len(value)


avg_capacity = {}
for key ,value in fid2capacity.items():
    avg_capacity [key] = sum(value)/len(value)
    
    
avg_permlanes = {}
for key ,value in fid2permlanes.items():
    avg_permlanes [key] = max(value)
    
    


In [153]:
#export dict as csv

In [154]:
df = pd.DataFrame.from_dict(avg_freespeed,orient='index')
df.to_csv("avg_freespeed.csv", encoding='utf-8',header=False)

In [155]:
df = pd.DataFrame.from_dict(avg_capacity,orient='index')
df.to_csv("avg_capacity.csv", encoding='utf-8',header=False)

In [156]:
df = pd.DataFrame.from_dict(avg_permlanes,orient='index')
df.to_csv("avg_permlanes.csv", encoding='utf-8',header=False)

In [157]:
df = pd.DataFrame.from_dict(fid2modes,orient='index')
df.to_csv("fid2modes.csv", encoding='utf-8',header=False)

In [158]:
df = pd.DataFrame.from_dict(fid2link,orient='index')
df.to_csv("fid2link.csv", encoding='utf-8',header=False)